# 特定の転写因子が結合するか否かの予測

転写因子の結合サイトに関連した実験が、ChIP-seq等を用いて、多数実施されるようになってきました。ここでは転写因子CTCFを対象として実施したChIP-seqの実験結果を用いて、特定の配列を与えた際に、CTCFが結合するか否かを予測するクラス分類問題を考えます。

ここでは、CTCFの結合の有無を機械学習的に調べるために、以下の様なセッティングを考えます。

* クラス分類問題を考える。つまり、何かをクラス0、何かをクラス1として、0と1を分類する問題とする。
* クラス1はCTCFが結合する配列である。学習データは、ChIP-seqのCTCFに関する実験で、ピークの中心周辺の配列で作成する。
    * 簡単のためピーク前後5０bp(合計100塩基）とします。
* クラス0はCTCFが結合しない配列であり、同一の実験で、ピークが無い領域の配列とする
    * 長さは揃えて１００塩基とします。
* これらのクラスを分ける分類器が作成できたら、任意の配列を入力として、結合するかしないかの判定をする機械の作成ができる。

<img src="img/DL_chipseq.png">

今回、ChIP-seqのデータからピークコールを行った後のデータが格納されている、[ChIP-Atlas](https://chip-atlas.org/) のデータを利用します。CTCFのChIP-seqを行った実験として、19人分のCTCFの結合サイトを解析した[Kasowski et al.](http://science.sciencemag.org/content/342/6159/750.long)の中の一つ、SRX356455 ( [ChIP-AtlasのSRX356455のページ](https://chip-atlas.org/view?id=SRX356455))　を利用します。ピークコール後の結果は、ChIP-AtlasのDownloadsから、ダウンロードできます。

ここでは、しきい値 p<1E-0.5 の結果から、ピークの前後１００塩基、および、ピークの無い位置からランダムに100塩基を取り出した前処理済みのデータをdata/sequences 以下に用意したので、これを用います。```SRX356455.05_peak_seq_100.txt```がピーク前後、```SRX356455.05_random_seq_100.txt```が、それ以外の領域から取ったものです。

## ライブラリの読み込み

学習は、配列の読み込みとネットワーク構造を除けば、特徴量からの学習と同様です。深層学習で利用するライブラリを読み込みます。

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models, transforms
import torch.utils.data as data
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

import os
import copy
import argparse
import time
import numpy as np
import pandas as pd

# データの読み込み

2節で酵母画像の特徴量から学習をした際と、同じ流れです。

配列解析に向けた大きな変更点は、塩基（ATGC）をそのまま学習に用いた場合、計算機にはただのアルファベットと思われてしまうので、これをベクトル情報に変換します。その方法は、酵母の芽の大きさの分類（no, medium,...） を0次元目、1次元目・・・と割当てた場合と同様で、Aを0次元目、Tを1次元目、としたベクトルを作成します。

In [2]:
SEQ_LENGTH = 100


def make_dataset(datadir):
    pos_seq = "SRX356455.05_peak_seq_100.txt"
    neg_seq = "SRX356455.05_random_seq_100.txt"
    # id      chr     start   end     seq
    data = pd.read_csv(os.path.join(datadir, "sequences", pos_seq), sep="\t")
    sequences = []
    classes = []
    for index, row in data[["id", "seq"]].iterrows():
        y = 1
        seq_vector = seq2vector(row["seq"])
        if len(seq_vector) == 0:
            continue
        sequences.append(seq2vector(row["seq"]))
        classes.append(np.array(y))
    data = pd.read_csv(os.path.join(datadir, "sequences", neg_seq), sep="\t")
    for index, row in data[["id", "seq"]].iterrows():
        y = 0
        seq_vector = seq2vector(row["seq"])
        if len(seq_vector) == 0:
            continue
        sequences.append(seq2vector(row["seq"]))
        classes.append(np.array(y))
    return sequences, classes


def seq2vector(seq):
    if type(seq) is not str: # Case on Null sequence
        return np.zeros((0,0))
    seq_array = np.zeros((4, SEQ_LENGTH))
    flag = 0
    for i in range(SEQ_LENGTH):
        s = seq[i]
        if s == "a" or s == "A":
            seq_array[0, i] = 1
        elif s == "c" or s == "C":
            seq_array[1, i] = 1
        elif s == "g" or s == "G":
            seq_array[2, i] = 1
        elif s == "t" or s == "T":
            seq_array[3, i] = 1
        else:
            flag += 1
    if len(seq) == flag: # Case on N sequence
        return np.zeros((0,0))
    seq_array = seq_array.astype(np.float32)
    return seq_array

データの読み込みも、特徴量からの学習時と同様です。

In [3]:
datadir = "data"

class DatasetFolder(data.Dataset):
    def __init__(self, X, y):
        self.samples = X
        self.targets = y
        self.transforms = transforms.Compose([
            ToTensorOfTarget()
        ])

    def __getitem__(self, index):
        sample = self.samples[index]
        sample = self.transforms(sample)
        target = self.targets[index]
        target = self.transforms(target)
        return sample, target

    def __len__(self):
        return len(self.samples)


class ToTensorOfTarget(object):
    def __call__(self, target):
        return torch.from_numpy(target)

# 全体を、training, valid, testに分ける。ここでは、3:1:1 に分割。
# training + valid が、機械学習の training data 相当。
X, y = make_dataset(datadir)
X_tmp, X_test, y_tmp, y_test = train_test_split(
    X, y, test_size = 0.20)
X_train, X_val, y_train, y_val = train_test_split(
    X_tmp, y_tmp, test_size = 0.25
)


sequence_datasets = {
    'train':DatasetFolder(X_train, y_train),
    'val':DatasetFolder(X_val, y_val),
    'test': DatasetFolder(X_test, y_test)
}

dataset_sizes = {x: len(sequence_datasets[x]) for x in ['train', 'val', 'test']}

## ネットワークモデルの定義

配列解析では、画像解析の様なゴールドスタンダードのモデルが存在する訳ではありません。特徴量抽出で行ったようなDense層を重ねることも一つですし、ローカルな特徴量を取るために画像同様Convoludionを利用する方法、更には、配列を時系列の音声処理の様にみなして解析を実施していく方法(WaveNet)も考えられます。

ここでは、前節の画像と同様のConvolutionを用いた方法で、シンプルにネットワークを構築します。
ネットワーク構成も、Conv -> Pooling -> Conv -> Pooling -> Dense -> Output という前節同様の構成を利用しています。
変更点は、画像が２次元情報だったのに対して、配列は1次元情報なので、1次元方法にのみConvolutionとPoolingを実施している点です。

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(4, 8, 3, padding=1) # 100
        self.pool = nn.MaxPool1d(2) # 50
        self.conv2 = nn.Conv1d(8, 16, 3, padding=1) # 50
        self.fc1 = nn.Linear(16 * 25 * 1, 20) # channel_num * x * y
        self.fc2 = nn.Linear(20, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # 100 -> 100 -> 50
        x = self.pool(F.relu(self.conv2(x))) # 50 -> 50 -> 25
        x = x.view(-1, 16 * 25 * 1) # channel_num * x * y
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

次に、学習ステップの詳細を定義します。

In [5]:
def print_test_accuracy(model, criterion, optimizer, phase):
    running_loss = 0.0
    running_corrects = 0
    model.train(False)

    for inputs, labels in dataloaders[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # 訓練のときだけ履歴を保持する
        with torch.set_grad_enabled(phase == 'train'):
            outputs = model(inputs)
            #_, classnums = torch.max(labels, 1)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

        # 統計情報
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels)

    # サンプル数で割って平均を求める
    epoch_loss = running_loss / dataset_sizes[phase]
    epoch_acc = running_corrects.double() / dataset_sizes[phase]
    print('On Test:\tLoss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))


def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    # 途中経過でモデル保存するための初期化
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    # 時間計測用
    end = time.time()

    print(model)
    print()

    for epoch in range(num_epochs):
        print('Epoch:{}/{}'.format(epoch, num_epochs - 1), end="")

        # 各エポックで訓練+バリデーションを実行
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # training mode
            else:
                model.train(False)  # evaluate mode

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # 訓練のときだけ履歴を保持する
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    #_, classnums = torch.max(labels, 1)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # 統計情報
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels)

            # サンプル数で割って平均を求める
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('\t{} Loss: {:.4f} Acc: {:.4f} Time: {:.4f}'.format(phase, epoch_loss, epoch_acc, time.time()-end), end="")

            # 精度が改善したらモデルを保存する
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            end = time.time()

        print()

    time_elapsed = time.time() - since
    print()
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val acc: {:.4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

バッチごとの読み込み設定をしましょう。

In [6]:
# バッチサイズ分のデータを読み込む。
# training はデータをシャッフルし、読み込み始める配列をランダムにする。
# 他はシャッフルの必要なし。
batch_size=64
workers=0
dataloaders = {
    'train': torch.utils.data.DataLoader(
        sequence_datasets['train'],
        batch_size=batch_size,
        shuffle=True,
        num_workers=workers),
    'val': torch.utils.data.DataLoader(
        sequence_datasets['val'],
        batch_size=batch_size,
        shuffle=False,
        num_workers=workers),
    'test': torch.utils.data.DataLoader(
        sequence_datasets['test'],
        batch_size=batch_size,
        shuffle=False,
        num_workers=workers)
}
dataset_sizes = {x: len(sequence_datasets[x]) for x in ['train', 'val', 'test']}

では、早速学習をしてみましょう。

In [8]:
device_name = "cpu"
epochs = 10
batch_size = 64
lr = 0.01
momentum = 0.9

device = torch.device(device_name)
model = Net()
model = model.to(device_name)

# 損失関数、最適化方法、学習率の更新方法を定義
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)

# 実際の学習を実施する
# 結果出力用ファイルのprefix
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=epochs)
# テストデータでの精度を求める
print_test_accuracy(model, criterion, optimizer, 'test')

Net(
  (conv1): Conv1d(4, 8, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(8, 16, kernel_size=(3,), stride=(1,), padding=(1,))
  (fc1): Linear(in_features=400, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=2, bias=True)
)

Epoch:0/9	train Loss: 0.5766 Acc: 0.6933 Time: 6.0771	val Loss: 0.5411 Acc: 0.7212 Time: 0.8388
Epoch:1/9	train Loss: 0.5245 Acc: 0.7387 Time: 4.9355	val Loss: 0.5114 Acc: 0.7481 Time: 0.8380
Epoch:2/9	train Loss: 0.4668 Acc: 0.7806 Time: 7.6239	val Loss: 0.4533 Acc: 0.7926 Time: 0.8236
Epoch:3/9	train Loss: 0.4306 Acc: 0.8034 Time: 4.8213	val Loss: 0.4166 Acc: 0.8133 Time: 0.8161
Epoch:4/9	train Loss: 0.4074 Acc: 0.8166 Time: 8.6646	val Loss: 0.4018 Acc: 0.8243 Time: 0.8111
Epoch:5/9	train Loss: 0.3906 Acc: 0.8259 Time: 4.6591	val Loss: 0.3900 Acc: 0.8303 Time: 0.8693
Epoch:6/9	train Loss: 0.3709 Acc: 0.8356 Time: 5.1666	val Loss: 0.3798 A

ここまでで、100塩基の配列を与えれば、CTCFが結合しそうか否かを判別することができそうな感じが見えてきます。一方で、CTCFの結合サイトの配列が何であるかは、ここからは分からないことが、深層学習がブラックボックスだと言われる所以となります。

機械学習を用いた配列解析は以上です。

今回の内容をまとめたソースコードは、```src/chipseq_prediction_cnn.py``` になります。
シェルから
```bash
$ python src/chipseq_prediction_cnn.py data -b 64 --lr 0.1 --epoch 20 -j 0
```
の様な形で実行可能です。

です。

# 参考：学習データの作成

* ここから先は、上記で利用している配列ファイルの作成方法なので、実行の必要はありません。（結構時間もかかります）

ChIP-atlasからのデータのダウンロードは、以下の様に行いました。
ChIP-atlasから、対象の実験(SRX356455)を検索します。

<img src="img/DL_chip_atlas_search.png">

次に、Downloadから、BEDファイルをダウンロードします。

<img src="img/DL_chip_atlas_dl.png">

ダウンロードしたBEDファイルは、```script```ディレクトリに入れておきます。

ChIP-AtlasからBED形式のピーク情報(SRX356455.05.bed など)をダウンロードして、```script```のディレクトリに入れます。
また、ChIP-Atlasは、現在は今回はヒトのhg19を利用しているので、ヒトゲノムの配列をダウンロードします。
```bash
$ cd script
$ mkdir ref_dir
$ cd ref_dir
$ wget -c http://hgdownload.cse.ucsc.edu/goldenPath/hg19/bigZips/chromFa.tar.gz
$ tar zxvf chromFa.tar.gz
$ cd ..
```

まず、BED形式から、ピーク位置情報を抽出します。
```bash
$ python get_chipseq_peak_info.py SRX356455.05.bed 100
```
これで、SRX356455.05_peak_extract.txt, SRX356455.05_peak_info.txtの２つのファイルが作成されます。それぞれ

* _peak_info.txt: 結合領域
* _peak_extract.txt: ピークの中心周辺100bpの領域

を表しています。
```bash
$ wc -l SRX356455.05_peak_extract.txt
   54613 SRX356455.05_peak_extract.txt
```
5.5万箇所くらいあることがわかります。

次に、ピーク以外の領域を取り出します。ここでは、5万配列を取り出します（ピークが5万箇所くらいあるので、その数に近い数を取り出します）。
```bash
$ python random_seq_except_listed.py ref_dir 100 50000 -l SRX356455.05_peak_info.txt -o SRX356455.05_random_100.txt
```
少々時間がかかります。選択された領域は、```SRX356455.05_random_100.txt```に格納されます。

最後に、作成した _peak_extract と、_ranom_100.txt のファイルを

```bash
$ python extract_seq.py SRX356455.05_peak_extract.txt ref_dir --id-column name > SRX356455.05_peak_seq_100.txt
$ python extract_seq.py SRX356455.05_random_100.txt ref_dir > SRX356455.05_random_seq_100.txt
```
これで、```SRX356455.05_peak_seq_100.txt```(ピーク周辺), ```SRX356455.05_random_seq.txt```(ピークの無いところ)の情報が得られます。前半は、このデータを利用しました。